<h1><b>Statistique en Bioinformatique : </b> TME 5 et 6 </h1>
<br>
L’objectif de ce TME est:
<br>
<ul>
<li> implémenter l'algorithme de Viterbi et l'estimation des paramètres (en utilisant le Viterbi training)
pour l'exemple du occasionally dishonest casino.   </li> 
</ul>
<br>
<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><b>Soumission</b></p>
<ul>
<li>Renomer le fichier TME5_6.ipynb pour NomEtudiant1_NomEtudiant2.ipynb </li>
<li>Soumettre via moodle </li>
</div>
</div>

Nom etudiant 1 : Antoine Grislain
<br>
Nom etudiant 2 : Simon Liétar
<br>

<h3>Introduction</h3>
Un casino parfois malhonnête (occasionally dishonest casino) utilise 2 types de pieces : fair et unfair. <br>
La matrice de transition entre les états cachés est:<br>

${\cal S}=\{F,U\}$ (fair, unfair):

$$
p = \left(
\begin{array}{cc}
0.99 & 0.01\\
0.05 & 0.95
\end{array}
\right)\ ,
$$

les probabilités d'émission des symboles 
${\cal O} = \{H,T\}$ (head, tail):
$$\begin{darray}{rcl}
e_F(H) =  0.5 &\ \ \ \ &
e_F(T) = 0.5 \nonumber\\
e_U(H) = 0.9 &\ \ \ \ &
e_U(T) = 0.1 \nonumber
\end{darray}$$

<br> Et la condition initiale $\pi^{(0)} = (0.999,0.001)$ (le jeux commence presque toujours avec le pieces juste (fair).

<b>Exercice 1</b>:
<u>Simulation</u>: Écrire une fonction qui simule $T$ jets de pièces. 
La fonction renverra un tableau à deux colonnes correspondant 
aux valeurs simulées pour les états cachés $X_t$ 
(type de dés utilisée, “F” ou “U”) et aux symboles observées $Y_t$ 
(résultat du jet de dés, “H” ou “T”). On simulera une séquence
de longueur 2000 qu'on gardera pour les applications ultérieures.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

#states
S = { 0:'F',1 :'U'}

#transition probability matrix
Pij = np.array([[0.99,0.01], [0.05,0.95]])

#emision symbols
O = {0:'H', 1: 'T'}

#emission probability matrix
Ei = np.array([[0.5,0.5], [0.9,0.1]]) #ça aurait dû être Eio

# initial Condition
pi0=np.array([0.999,0.001])

#number of jets
T = 2000

In [2]:
import random

# Fonction qui simule T jets de pieces
def jets(T: int, pi0: np.ndarray, Eij: np.ndarray, Pij: np.ndarray):
    """
      simulation of occasionally dishonest casino
      input1 T: number of jets
      input2 pi0: initial condition
      input3 Eij: emission probability matrix
      input4 Pij: transition probability matrix
      output1 jetsRes: matrix |2xT| containing simulations
    """

    # Creation du tableau
    output = np.zeros((T, 2), dtype=int)
    state = None

    for it in range(T):
      state = np.random.choice(np.arange(pi0.shape[0]), p=(Pij[state, :] if state is not None else pi0))

      output[it, 0] = state
      output[it, 1] = np.random.choice(np.arange(Eij.shape[1]), p=Eij[state, :])

    return output


def printSimulation(resultat):
    for i in resultat :
        print (S[i[0]], O[i[1]])

jetsRes = jets(T, pi0, Ei, Pij)
printSimulation(jetsRes)

F H
F H
F H
F T
F H
F T
F T
F T
F H
F H
F H
F H
F H
F H
F H
F T
F H
F T
F H
F T
F T
F T
F T
F H
F H
F T
F T
F H
F H
F T
F H
F H
F H
F H
F H
F H
F H
F T
F T
F H
F H
F H
F H
F H
F T
F T
F T
F H
F T
F H
F T
F H
F T
F H
F H
F H
F T
F T
F H
F H
F T
F H
F H
F H
F H
F T
F H
F T
F T
F H
F H
F T
F T
F H
F H
F H
F T
F H
F T
F H
F H
F H
F T
F H
F H
F T
F H
F T
F H
F H
F T
F H
F T
F T
F T
F T
F H
F T
F H
F T
F T
F T
F H
F H
F T
F H
F T
F H
F T
F T
F T
F T
F T
F T
F T
F H
F T
F H
F H
F T
F H
F H
F T
F T
F T
F T
F H
F H
F T
F T
F T
F T
F T
F H
F T
F T
F T
F T
F T
F H
F T
F H
F H
F T
F T
F T
F H
F H
F H
F H
F H
F T
F H
F T
F H
F T
F H
F H
F H
F H
F T
F H
F H
F H
F T
F T
F T
F H
F H
F H
F T
F H
F T
F T
F T
F H
F H
F H
F T
F T
F H
F H
F H
F T
F T
F T
U H
U H
U H
F T
F T
F H
F H
F T
F T
F T
F T
F T
F H
F H
F T
F H
F T
F H
F T
F H
F T
F T
F H
F H
F H
F H
F T
F T
F T
F T
F H
F T
F T
F H
F T
F T
F H
F T
F H
F H
F H
F H
F T
F H
F H
F H
F T
F T
F H
F H
F T
F T
F T
F T
F H
F H
F T
F T
F T
F T
F T
F T
F H
F T


<b>Exercice 2</b>: <u>Algorithme de Viterbi </u>: Écrire une fonction qui permet
de déterminer la séquence $(i^\star_t)_{t=0:T}$ d'états cachés
plus probable, ainsi que sa probabilité. Pour tester votre fonction utiliser le résultat de la 
simulation (2éme colonne) de la question 1. Comparer $(i^\star_t)_{t=0:T}$ avec
les vrais états cachés (1ère colonne de la simulation).


In [3]:
# Algorithme de Viterbi
import operator

def viterbi(jets: np.ndarray, Pij: np.ndarray, Ei: np.ndarray, pi0: np.ndarray, nS: int, nO: int, enLog: bool):
    """
    Implement Viterbi algorithm
    input1 jets: matrix |2xT| containing simulations
    input4 Pij: transition probability matrix
    input3 Ei: emission probability matrix
    input4 pi0: initial condition
    input5 nS: number of states
    input6 nO: number of observations
    input7 enLog: bool, when True we apply log to avoid underflow
    output1 i_star: most problable path
    output2 prob: probability associated to the most problable path
    """

    obs = jets[:, 1]
    T = len(obs) # Nombre d'observations (longueur des observations)

    delta = np.empty((nS, T))
    psi = np.empty((nS, T), dtype=int)

    if enLog:
        delta[:, 0] = np.log(pi0) + np.log(Ei[:, obs[0]])
    else:
        delta[:, 0] = pi0 * Ei[:, obs[0]]

    for t in range(1, T):
        if enLog:
            a = delta[:, t - 1] + np.log(Pij.T) + np.log(Ei[:, obs[t]])
        else:
            a = delta[:, t - 1] * Pij.T * Ei[:, obs[t]]

        delta[:, t] = a.max(axis=1)
        psi[:, t] = a.argmax(axis=1)

    sequence = np.empty(T, dtype=int)

    prob = delta[:, -1].max()
    sequence[-1] = delta[:, -1].argmax()

    for t in range(T - 2, -1, -1):
        sequence[t] = psi[sequence[t + 1], t + 1]

    return sequence, prob

def analyseResultats(jets: np.ndarray, estimation: np.ndarray):
    """
    Compare expected and obtained paths
    input1 jets: expected path
    input2 estimation: obtained path
    output1 error: percentage error
    """

    return (jets[:, 0] != estimation).sum() / jets.shape[0] * 100.0

i_est, P_est = viterbi(jetsRes, Pij, Ei, pi0, 2, 2, True)
error = analyseResultats(jetsRes, i_est)
print('erreur d\'estimation de viterbi:')
print(error,'%')
print('Probabilité estimé:')
print(P_est)

erreur d'estimation de viterbi:
6.800000000000001 %
Probabilité estimé:
-1336.3545319796713


<b>Exercice 3</b>: <u>Estimation des paramètres</u>
<br>
3.1) Écrire une fonction qu'utilise tous les résultats de la simulation
(états et symboles) pour compter les nombres d'occurrence $N_{ij}$ est $M_{iO}$ définis en cours. Estimer $P_{ij}$ est $E_i(O)$. Attention, pour éviter les probabilités à zéro nous allons utiliser les pseudo-count.

In [4]:
# Estimation de Parametres par contage
def nombresOccurrence(jets: np.ndarray, nS: int, nO: int):
    """
    Parameter estimation
    input1 jets: matrix |2xT| containing data
    input2 nS: number of states
    input3 nO: number of observations
    output1 Nij: transition probability matrix
    output2 Mio: emission probability matrix
    output3 pi0: initial condition
    """

    Nij = np.ones((nS,nS), dtype=int) #pseudo-count = 1
    Mio = np.ones((nS,nO), dtype=int)  #pseudo-count = 1
    pi0 = np.ones((nS), dtype=int)

    pi0[jets[0, 0]] += 1

    for t in range(1, jets.shape[0]):
        Nij[jets[t - 1, 0], jets[t, 0]] += 1
        Mio[jets[t, 0], jets[t, 1]] += 1

    return (
        Nij / Nij.sum(axis=1, keepdims=True),
        Mio / Mio.sum(axis=1, keepdims=True),
        pi0 / pi0.sum()
    )

Nij, Mio, pi0 = nombresOccurrence(jetsRes, 2, 2)

print('Pij estimé:')
print(Nij)
print('\nEio estimé:')
print(Mio)
print('\npi0 estimé:')
print(pi0)

Pij estimé:
[[0.98848485 0.01151515]
 [0.05382436 0.94617564]]

Eio estimé:
[[0.50484848 0.49515152]
 [0.90368272 0.09631728]]

pi0 estimé:
[0.66666667 0.33333333]


3.2) <u> Viterbi training </u>: Écrire une fonction qui utilise
seulement la séquence $(O_t)_{t=0:T}$ (2emme colonne de la simulation) pour estimer les
paramètres $P_{ij}$ est $Ei(O)$. On s’arrêtera quand les différences entre les logVraissamblance est inférieur à 1e-04. Comparer les résultats de 3.1 et de 3.2 (3.2 avec plusieurs restarts,
et avec initialisation des paramètres aléatoire).


In [5]:
import matplotlib.pyplot as plt

# Initialisation aleatoire de Pij, Eij, pi0
def InititRandom(nS: int, nO: int):
    """
    randomly initialisations
    input1 nS: number of states
    input2 nO: number of observations
    output1 Pij_init: transition probability matrix
    output2 Ei_init: emission probability matrix
    output3 pi0_init: initial condition
    """
    random.seed(10)

    Pij_init = np.random.rand(nS, nS)
    Ei_init = np.random.rand(nS, nO)
    pi0_init = np.random.rand(nS)

    return (
        Pij_init / Pij_init.sum(axis=1, keepdims=True),
        Ei_init / Ei_init.sum(axis=1, keepdims=True),
        pi0_init / pi0_init.sum()
    )

# Calcule log Vraissamblance
def logLikelhihood(Aij: np.ndarray, Bij: np.ndarray, pi0: np.ndarray, jets: np.ndarray):
    """
    compute Log Likelihood
    input1 Pij: transition probability matrix
    input2 Ei: emission probability matrix
    input3 pi0: initial condition
    input4 jets: matrix |2xT| containing data
    """
    etat_seq = jets[:,0]
    obs_seq = jets[:,1]
    T = len(obs_seq) # Nombre d'observations (longueur des observations)
    lLikelihood = 0.0

    for t in range(T):
        lLikelihood += np.log(Aij[etat_seq[t - 1], etat_seq[t]] if t > 0 else pi0[etat_seq[t]])
        lLikelihood += np.log(Bij[etat_seq[t], obs_seq[t]])

    return lLikelihood


def Training(jets: np.ndarray, nS: int, nO: int):
    """
    Viterbi Training
    input1 jets: matrix |2xT| containing data
    input2 nS: number of states
    input3 nO: number of observations
    output1 Pij_est: transition probability matrix
    output2 Ei_est: emission probability matrix
    output3 pi0_est: initial condition
    output4 lLikelihood: log Likelihood
    """
    jets_est = np.array(jets)
    Pij_est, Ei_est, pi0_est = InititRandom(nS, nO)

    max_iteration_count = 10000
    criterion = 1e-04
    lLikelihood = np.zeros((max_iteration_count))

    it = 0

    for it in range(max_iteration_count):
        states, ll = viterbi(jets, Pij_est, Ei_est, pi0_est, nS, nO, True)

        Pij_est, Ei_est, pi0_est = nombresOccurrence(np.array([states, jets_est[:, 1]]).T, nS, nO)
        lLikelihood[it] = logLikelhihood(Pij_est, Ei_est, pi0_est, jets_est)

        if it > 0 and (lLikelihood[it] - lLikelihood[it - 1]) < criterion:
            break

    return Pij_est, Ei_est, pi0_est, lLikelihood[:(it + 1)]

#imprimer les Parametres du Viterbi Training
Pij_est, Ei_est, pi0_est, lLikelihood = Training(jetsRes, 2, 2)
itCount = len(lLikelihood)
print('Le modèle est convergé après '+str(itCount)+' itérations.')
print('\nPij estimée:')
print(Pij_est)
print('\nEij estimée:')
print(Ei_est)

Le modèle est convergé après 2 itérations.

Pij estimée:
[[0.0010627  0.9989373 ]
 [0.88418079 0.11581921]]

Eij estimée:
[[0.70957447 0.29042553]
 [0.45625588 0.54374412]]


<font color="blue">
Remark:
</font>

L'algorithme converge très rapidement mais on est assez loin des paramètres réels.

3.3) <u>Viterbi training deuxième version</u>. 
<BR>Écrivez une version de 3.2 qui:
- part plusieurs fois (100x) d'une initialisation aléatoire desparamètres de l'HMM,
- utilise Viterbi training pour estimer les paramètres,
- calcul la log-vraisemblance pour les paramètres estimés,
- sauvegarde seulement l'estimation avec la valeur maximale de lalog-vraisemblance.

Qu'est-ce que vous observez?



In [6]:
# Viterbi Training  deuxiemme version

def TrainingV2(jets: np.ndarray, nS: int, nO: int, nIterat: int = 100):
    """
    Viterbi Training version 2.0
    input1 jets: matrix |2xT| containing data
    input2 nS: number of states
    input3 nO: number of observations
    input4 nIterat: number of iterations
    output1 Pij_best: best transition probability matrix
    output2 Ei_best: best emission probability matrix
    output3 pi0_best: best initial condition
    output4 lLikelihood_best: best log Likelihood
    """

    best_params: tuple[np.ndarray, np.ndarray, np.ndarray] | None = None
    lLikelihood_best = -np.inf

    for x in range(nIterat):
        *params, ll = Training(jets, nS, nO)

        if ll[-1] > lLikelihood_best:
            best_params = tuple(params)
            lLikelihood_best = ll[-1]

    assert best_params is not None
    return *best_params, lLikelihood_best


# Imprimer les Parametres du Viterbi Training deuxiemme version
nIterat = 100
Pij_best, Ei_best, pi0_best, lLikelihood_best = TrainingV2(jetsRes, 2, 2, nIterat)

print('Meilleur Pij estimée:');
print(Pij_best)
print('\nMeilleur Eij estimée:')
print(Ei_best)

Meilleur Pij estimée:
[[0.99529965 0.00470035]
 [0.02325581 0.97674419]]

Meilleur Eij estimée:
[[0.51734274 0.48265726]
 [0.90066225 0.09933775]]


<font color="blue">
Remark:
</font>

On est beaucoup plus proche des paramètres réels.